In [1]:
import math
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import random

# Use log not ln
# Label axes, also with units

Dmin = math.log(278, 10) # lower bound for diffusivity
Dmax = math.log(40000, 10) # upper bound for diffusivity
phi = 0 # phase

def solution(x, t, A, omega, D, phi):
  """This is the function which gives back the hydrolic head
  omega: frequency
  A: amplitude
  x: distance
  t: time"""
  sinus = math.sin(-x*math.sqrt(omega/(2*D))+ omega*t + phi)
  ret = A*math.exp(-x*math.sqrt(omega/(2*D)))*sinus
  return ret 

In [2]:
# Create figure
fig = go.Figure()

# Fixed parameters
A = 2
psi = 0
omega = 2
D = 1000

# Add traces one for each time step
for step in np.arange(0, 5, 0.05):
    fig.add_trace(
        go.Scatter(
            visible=False,
            line=dict(color="#00CED1", width=6),
            name="𝜈 = " + str(step),
            x=np.arange(0, 100, 1),
            y=[solution(d, step, A, omega, D, phi) for d in np.arange(0, 100, 1)]))

fig.data[10].visible = True

# Create and add slider
steps = []
for i in range(len(fig.data)):
    step = dict(
        method="update",
        args=[{"visible": [False] * len(fig.data)},
              {"title": "Slider switched to step: " + str(i)}],  # layout attribute
    )
    step["args"][0]["visible"][i] = True  # Toggle i'th trace to "visible"
    steps.append(step)

sliders = [dict(
    active=10,
    currentvalue={"prefix": "Timestep: "},
    pad={"t": 50},
    steps=steps
)]

fig.update_layout(
    sliders=sliders
)

fig.show()


In [6]:
dist = [1, 10, 100] # given in meters
timesteps = [7, 30, 180] # given in days

# Initialize empty return list
result = [[ [None for _ in range(100) ] for _ in range(3)] for _ in range(3)]

for i, t in enumerate(timesteps): 
  for j, x in enumerate(dist):
    for w in range(100):
      A = 10**(2*random.random() - 1) # sample amplitude from exponential of uniform distribution
      omega = (math.pi - 2*math.pi/7)*random.random() + 2*math.pi/7 # sample frequency from uniform distribution
      D = 10**((Dmax - Dmin)*random.random() + Dmin)

      result[i][j][w] = solution(x, t, A, omega, D, phi)

In [7]:
fig = make_subplots(rows=3, cols=3, subplot_titles=('X = 1m, t = 7d','X = 10m, t = 7d', 'X = 100m, t = 7d','X = 1m, t = 30d', 'X = 10m, t = 30d','X = 100m, t = 30d', 'X = 1m, t = 180d','X = 10m, t = 180d', 'X = 100m, t = 180d')) # Plot histograms

fig.add_trace(go.Histogram(x=result[0][0]), row=1, col=1)
fig.add_trace(go.Histogram(x=result[0][1]), row=1, col=2)
fig.add_trace(go.Histogram(x=result[0][2]), row=1, col=3)

fig.add_trace(go.Histogram(x=result[1][0]), row=2, col=1)
fig.add_trace(go.Histogram(x=result[1][1]), row=2, col=2)
fig.add_trace(go.Histogram(x=result[1][2]), row=2, col=3)

fig.add_trace(go.Histogram(x=result[2][0]), row=3, col=1)
fig.add_trace(go.Histogram(x=result[2][1]), row=3, col=2)
fig.add_trace(go.Histogram(x=result[2][2]), row=3, col=3)

fig.update_layout(showlegend=False)
# Change y axis range

In [9]:
# Sensitivity analysis regarding A
import plotly.express as px

# Set other parameters to mean
omega = (math.pi + 2*math.pi/7)/2
D = (10**(Dmin)+10**(Dmax)/2)

A_range = list(np.linspace(10**(-1), 10, 100))
fig = go.Figure()
for i, t in enumerate(timesteps): 
  for j, x in enumerate(dist):
      func_value = [solution(x, t, A, omega, D, phi) for A in A_range]
      fig.add_traces(go.Scatter(x=A_range, y=func_value, mode='lines', name = 'Hydraulic head in dependance of A, X='+str(x)+', t='+str(t)))

fig.show()

In [10]:
A_range = list(np.linspace(10**(-1), 10, 100))
fig = go.Figure()
for i, t in enumerate(timesteps): 
  for j, x in enumerate(dist):
      func_value = [solution(x, t, A, omega, D, phi) for A in A_range]
      dx = A_range[1]-A_range[0]
      dydx = np.gradient(func_value, dx)
      fig.add_traces(go.Scatter(x=A_range, y=dydx, mode='lines', name = 'Derivative of analytical solution w.r.t A, X='+str(x)+', t='+str(t)))

fig.show()

In [14]:
# Only vary A

dist = [1, 10, 100] # given in meters
timesteps = [7, 30, 180] # given in days

# Initialize empty return list
result = [[ [None for _ in range(100) ] for _ in range(3)] for _ in range(3)]


# Set other parameters to mean
omega = (math.pi + 2*math.pi/7)/2
D = (10**(Dmin)+10**(Dmax)/2)

for i, t in enumerate(timesteps): 
  for j, x in enumerate(dist):
    for w in range(100):
      A = 10**(2*random.random() - 1) # sample amplitude from exponential of uniform distribution
      result[i][j][w] = solution(x, t, A, omega, D, phi)

fig = make_subplots(rows=3, cols=3, subplot_titles=('X = 1m, t = 7d','X = 10m, t = 7d', 'X = 100m, t = 7d','X = 1m, t = 30d', 'X = 10m, t = 30d','X = 100m, t = 30d', 'X = 1m, t = 180d','X = 10m, t = 180d', 'X = 100m, t = 180d')) # Plot histograms

fig.add_trace(go.Histogram(x=result[0][0]), row=1, col=1)
fig.add_trace(go.Histogram(x=result[0][1]), row=1, col=2)
fig.add_trace(go.Histogram(x=result[0][2]), row=1, col=3)

fig.add_trace(go.Histogram(x=result[1][0]), row=2, col=1)
fig.add_trace(go.Histogram(x=result[1][1]), row=2, col=2)
fig.add_trace(go.Histogram(x=result[1][2]), row=2, col=3)

fig.add_trace(go.Histogram(x=result[2][0]), row=3, col=1)
fig.add_trace(go.Histogram(x=result[2][1]), row=3, col=2)
fig.add_trace(go.Histogram(x=result[2][2]), row=3, col=3)

fig.update_layout(showlegend=False)


# Interesting to note: 
# Histograms are very imbalanced because we sample the log

In [15]:
# Sensitivity analysis regarding omega

# Set other parameters to mean
A = (math.exp(-1)+ math.exp(1))/2
D = (math.exp(Dmin)+math.exp(Dmax)/2)

omega_range = list(np.linspace(2*math.pi/7, math.pi, 100))


fig = go.Figure()
for i, t in enumerate(timesteps): 
  for j, x in enumerate(dist):
      func_value = [solution(x, t, A, omega, D, phi) for omega in omega_range]
      fig.add_traces(go.Scatter(x=omega_range, y=func_value, mode='lines', name = 'Hydraulic head in dependance of omega, X='+str(x)+', t='+str(t)))

fig.show()

In [16]:
fig = go.Figure()
for i, t in enumerate(timesteps): 
  for j, x in enumerate(dist):
      func_value = [solution(x, t, A, omega, D, phi) for omega in omega_range]
      dx = omega_range[1]-omega_range[0]
      dydx = np.gradient(func_value, dx)
      fig.add_traces(go.Scatter(x=omega_range, y=dydx, mode='lines', name = 'Derivative of analytical solution w.r.t omega, X='+str(x)+', t='+str(t)))

fig.show()

In [17]:
# Only vary omega

dist = [1, 10, 100] # given in meters
timesteps = [7, 30, 180] # given in days

# Initialize empty return list
result = [[ [None for _ in range(100) ] for _ in range(3)] for _ in range(3)]

# Set other parameters to mean
A = (10**(-1)+ 10)/2
D = (10**(Dmin)+10**(Dmax)/2)


for i, t in enumerate(timesteps): 
  for j, x in enumerate(dist):
    for w in range(100):
      omega = (math.pi - 2*math.pi/7)*random.random() + 2*math.pi/7 # sample frequency from uniform distribution

      result[i][j][w] = solution(x, t, A, omega, D, phi)


fig = make_subplots(rows=3, cols=3, subplot_titles=('X = 1m, t = 7d','X = 10m, t = 7d', 'X = 100m, t = 7d','X = 1m, t = 30d', 'X = 10m, t = 30d','X = 100m, t = 30d', 'X = 1m, t = 180d','X = 10m, t = 180d', 'X = 100m, t = 180d')) # Plot histograms

fig.add_trace(go.Histogram(x=result[0][0]), row=1, col=1)
fig.add_trace(go.Histogram(x=result[0][1]), row=1, col=2)
fig.add_trace(go.Histogram(x=result[0][2]), row=1, col=3)

fig.add_trace(go.Histogram(x=result[1][0]), row=2, col=1)
fig.add_trace(go.Histogram(x=result[1][1]), row=2, col=2)
fig.add_trace(go.Histogram(x=result[1][2]), row=2, col=3)

fig.add_trace(go.Histogram(x=result[2][0]), row=3, col=1)
fig.add_trace(go.Histogram(x=result[2][1]), row=3, col=2)
fig.add_trace(go.Histogram(x=result[2][2]), row=3, col=3)

fig.update_layout(showlegend=False)
# Interesting to note:
# Histograms are very spreaded because of the periodic function values

In [18]:
# Sensitivity analysis for D

# Set other parameters to mean
A = (math.exp(-1)+ math.exp(1))/2
omega = (math.pi + 2*math.pi/7)/2

D_range = list(np.linspace(10**(Dmin), 10**(Dmax), 100))

fig = go.Figure()
for i, t in enumerate(timesteps): 
  for j, x in enumerate(dist):
      func_value = [solution(x, t, A, omega, D, phi) for D in D_range]
      fig.add_traces(go.Scatter(x=D_range, y=func_value, mode='lines', name = 'Hydraulic head in dependance of D, X='+str(x)+', t='+str(t)))

fig.show()

In [19]:
dydx = np.gradient(func_value, dx)

fig = go.Figure()
for i, t in enumerate(timesteps): 
  for j, x in enumerate(dist):
      func_value = [solution(x, t, A, omega, D, phi) for D in D_range]
      dx = D_range[1]-D_range[0]
      dydx = np.gradient(func_value, dx)
      fig.add_traces(go.Scatter(x=omega_range, y=dydx, mode='lines', name = 'Derivative of analytical solution w.r.t D, X='+str(x)+', t='+str(t)))

fig.show()

In [20]:
# Only vary D

dist = [1, 10, 100] # given in meters
timesteps = [7, 30, 180] # given in days

# Initialize empty return list
result = [[ [None for _ in range(100) ] for _ in range(3)] for _ in range(3)]

A = (math.exp(-1)+ math.exp(1))/2
omega = (math.pi + 2*math.pi/7)/2

for i, t in enumerate(timesteps): 
  for j, x in enumerate(dist):
    for w in range(100):
      D = math.exp((Dmax - Dmin)*random.random() + Dmin)

      result[i][j][w] = solution(x, t, A, omega, D, phi)


fig = make_subplots(rows=3, cols=3, subplot_titles=('X = 1m, t = 7d','X = 10m, t = 7d', 'X = 100m, t = 7d','X = 1m, t = 30d', 'X = 10m, t = 30d','X = 100m, t = 30d', 'X = 1m, t = 180d','X = 10m, t = 180d', 'X = 100m, t = 180d')) # Plot histograms

fig.add_trace(go.Histogram(x=result[0][0]), row=1, col=1)
fig.add_trace(go.Histogram(x=result[0][1]), row=1, col=2)
fig.add_trace(go.Histogram(x=result[0][2]), row=1, col=3)

fig.add_trace(go.Histogram(x=result[1][0]), row=2, col=1)
fig.add_trace(go.Histogram(x=result[1][1]), row=2, col=2)
fig.add_trace(go.Histogram(x=result[1][2]), row=2, col=3)

fig.add_trace(go.Histogram(x=result[2][0]), row=3, col=1)
fig.add_trace(go.Histogram(x=result[2][1]), row=3, col=2)
fig.add_trace(go.Histogram(x=result[2][2]), row=3, col=3)

fig.update_layout(showlegend=False)
# Interesting to note:
# Histograms don't take a wide range of function values because 
# of the low sensitivity